In [ ]:
import ollama
import os
import pandas as pd

device = "RTX4060"
batch_sizes = [1,8,32,128]
outputs = []

In [ ]:
# warmup
# context 32K
# Internet Access NO

def bench(prompts,prefix = "", stream = False):
    result = {}
    output = ollama.chat(
        model="llama3",
        messages=[
            {"role": "user", "content": prefix},
        ] + [{"role": "user", "content": prompt+"\n"} for prompt in prompts],
        stream = stream
    )

    result["prompts"] = len(prompts)
    result["input_tokens"] = output.prompt_eval_count
    result["output_tokens"] = output.eval_count
    result["total_time"] = output.total_duration/1e+6
    result["load_time"] = output.load_duration/1e+6
    result["prompt_eval_time"] = output.prompt_eval_duration/1e+6
    result["output_eval_time"] = output.eval_duration/1e+6

    return result, output.message.content

bench(["Do you know whos dashing, smart and disappears in a flash"])

In [ ]:
files = os.listdir("data/prompts/")

# iterating over different files full of variations of questions
for filename in files:
    # getting some data about the question out of the name of the file
    _, questions, prompt_complexity, prompt_length = filename.split(".")[0].split("_")

    # iterating over different batch sizes
    for batch_size in batch_sizes:
        with open(f"./data/prompts/{filename}", "r") as file:
            all_prompts = file.read().split("\\n")

            for i in range(len(all_prompts)):
                all_prompts[i] = f"Question {i+1} : {all_prompts[i].strip()}"
        # iterating over the prompts in the files in batches of batch_size
        for i in range(0, len(all_prompts)-1, batch_size):
            prompts = all_prompts[i: i+batch_size]
            # iterating over different prefix expecting the output length to be different
            for output_length in ["short", "long"]:
                if output_length == "short":
                    benchmark, answer = bench(prompts, prefix = f"give very short answers to the questions, if answers get longer than 50 words stop")
                else:
                    benchmark, answer = bench(prompts, prefix = f"give long answers to the questions, if answers get longer than 500 words stop")
                # saving
                with open(f"./data/answers/answer_{batch_size}_{prompt_complexity}_{prompt_length}.md", "w", encoding="utf-8") as file:
                    file.write(answer)

                benchmark["prompt_complexity"] = prompt_complexity
                benchmark["prompt_length"] = prompt_length
                benchmark["output_length"] = output_length
                benchmark["device"] = device

                outputs.append(benchmark)
                outputs_df = pd.DataFrame(outputs)
                outputs_df.to_csv(f"./data/{device}_outputs.csv", index = False)

In [ ]:
outputs_df = pd.read_csv(f"./data/{device}_outputs.csv")
outputs_df